# California 2017 Fire recovery 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
from shapely.geometry import Point, LineString, MultiPolygon, asMultiPolygon, Polygon
from shapely import wkb, wkt
import shapely
import geopandas as gpd
from shapely.ops import unary_union
import requests
from bs4 import BeautifulSoup
import re
import os
import zipfile
import wget
from datetime import datetime
from multiprocessing.dummy import Pool as ThreadPool 
import geocoder
import geopy
%matplotlib inline

import sys
# sys.path.insert(0, '/Users/jianglongli/Desktop/workbook/Freddie_project/PostGIS/gisfeaturecode_v7/')
from mapping_utility_v2 import map_geopandas, map_AllHouses
from mapping_utility_fire import map_geopandas_fire
from python_postgis_talk_utility import transform_pd_to_gpd_general, transform_pd_to_gpd
from cali_fire_utility import geomatch, readin_shapefile, timer, fire_postprocessing, create_fire_union
from cali_fire_utility import download_and_create_shp, download_read_curent_fire
from cali_fire_utility import map_fires, geocode, multigeocoding

pd.options.display.max_columns = 100

In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
url_cali = "https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/current_year_fire_data/California/"
url_master = "https://rmgsc.cr.usgs.gov"
folder_root = '/Users/jianglongli/Desktop/workbook/data/disaster_recovery'
folder_sub = 'cali_fire'

### read in hve turned off file and try matching

In [4]:
hve_raw = pd.read_csv('%s/cali_turnedoff/cali_turned_off.csv' % folder_root, dtype={'zip': str})
# hve_raw = pd.read_csv('%s/cali_turnedoff/cali_turned_off_geocoded.csv' % folder_root, dtype={'zip': str}) # geocoded
print('raw hve count: %s' % hve_raw.shape[0])

hve_raw.loc[:, 'long'] = hve_raw['long'].apply(lambda x: -x if x>0 else x)
hve = hve_raw[hve_raw.lat.notnull()]
print('valid lat/lng count: %s' % hve.shape[0])

hve_null = hve_raw[hve_raw.lat.isnull()]
hve_null.loc[:, 'address'] = hve_null.apply(lambda row: row['address'] + ',' + ' CA ' + row['zip'], axis=1)
sample = hve_null.sample(100)
print('null lat/lng hve: %s' % hve_null.shape[0])

raw hve count: 359266
valid lat/lng count: 324931
null lat/lng hve: 34335


### geocoding hve with missing lat/lng

In [28]:
# %%time
# hve_geocode = multigeocoding(hve_null, 60, 6)

In [29]:
# hve_geo = hve_geocode[hve_geocode.geocd.notnull()]
# hve_geo.loc[:, 'lat'] = hve_geo.geocd.apply(lambda x: x[0])
# hve_geo.loc[:, 'long'] = hve_geo.geocd.apply(lambda x: x[1])
# hve_geo = hve_geo.drop('geocd', axis=1)
# hve_comb = pd.concat([hve, hve_geo])

In [31]:
# %store -r hve_geo
# hve_comb = pd.concat([hve, hve_geo])

### download shp files and create total geopandas dataframe

In [5]:
%%time
gdf = download_and_create_shp(url_cali, url_master, folder_root, folder_sub, 
                              download_type='zip', unzip=True, verb=True)


########## scraping start: 22:54:55 ##########
totally 1558 zip files url parsed! 22:56:11

########## check unzipped files: 22:56:11 ##########
CPU times: user 48.6 s, sys: 1.36 s, total: 49.9 s
Wall time: 2min 3s


In [6]:
gdf = fire_postprocessing(gdf)

Ring Self-intersection at or near point -122.29322609958997 38.368854461891438
Ring Self-intersection at or near point -122.29322609958997 38.368854461891438
Ring Self-intersection at or near point -122.29322609958997 38.368854461891438
Ring Self-intersection at or near point -122.29322609959007 38.368854461891388
Ring Self-intersection at or near point -122.29322609958997 38.368854461891438
Ring Self-intersection at or near point -120.99716398229684 41.335605047406816
Ring Self-intersection at or near point -120.03658782922585 37.62942306856656
Ring Self-intersection at or near point -120.0365878176455 37.62942307763192
Ring Self-intersection at or near point -120.0365878288182 37.62942306858718
Ring Self-intersection at or near point -120.0365878176455 37.62942307763192
Ring Self-intersection at or near point -120.0365878288182 37.62942306858718
Ring Self-intersection at or near point -120.0365878176455 37.62942307763192
Ring Self-intersection at or near point -123.62637115611462 41.

In [7]:
%%time
gdf_union = create_fire_union(gdf, 6) 

CPU times: user 23min 1s, sys: 14.5 s, total: 23min 15s
Wall time: 13min 54s


In [27]:
%%time
print(timer())
result = geomatch(hve, gdf_union, 'firename')

10:12:05
CPU times: user 18min 33s, sys: 2.08 s, total: 18min 35s
Wall time: 18min 37s


**download current fire shapefile**

In [261]:
gdf_current = download_read_curent_fire(folder_root)
gdf_current_ca = gdf_current[gdf_current.state == 'CA']
gdf_current_ca = fire_postprocessing(gdf_current_ca)
result_current = geomatch(hve, gdf_current_ca, 'firename', ['firename', 'geometry', 'perdattime'])

### fire impact reporting

+ **fire this year**

In [9]:
counts = result.firename.value_counts()
summary = pd.concat([counts.rename('count'), gdf_union.set_index('firename')], axis=1, join='inner')
summary = summary.reset_index().rename(columns={'index': 'firename'})
summary = transform_pd_to_gpd(summary, geometry='geometry')

In [284]:
summary.to_csv(folder_root + '/' + 'fire_summary.csv', index=False)

In [321]:
map_geopandas_fire(summary, ckeep=['firename', 'geometry', 'count'], clabel='firename', 
              cpop=['firename', 'count'], saveTo='./', saveName='2017fire', saveOnly=True)

In [275]:
hve_onfire = result[result.firename.notnull()]
hve_onfire.drop('geometry', axis=1).to_csv('/Users/jianglongli/Desktop/workbook/data/hve_onfire.csv', index=False)

+ **current fire**

In [325]:
hve_onfire_current = result_current[result_current.firename.notnull()]
summary_crrt = hve_onfire_current.firename.value_counts().rename('count')
summary_crrt = pd.concat([summary_crrt, gdf_current_ca[['firename', 'gisacres', 'geometry']].set_index('firename')], 
         axis=1, join='inner').reset_index().rename(columns={'index': 'firename'})
summary_crrt = transform_pd_to_gpd(summary_crrt, geometry='geometry')
summary_crrt.to_csv('/Users/jianglongli/Desktop/workbook/data/summary_crrt.csv', index=False)

In [328]:
map_geopandas_fire(summary_crrt, ckeep=['firename', 'geometry', 'count'], clabel='firename', 
              cpop=['firename', 'count'], saveTo='./', saveName='current_fire', saveOnly=True)

map_geopandas_fire(gdf[gdf.firename == 'ABNEY'], ckeep=['firename', 'geometry'], clabel='firename', 
              cpop=['firename'], saveTo='./', saveName='before_union', saveOnly=True)

map_geopandas_fire(gdf_union[gdf_union.firename == 'ABNEY'], ckeep=['firename', 'geometry'], clabel='firename', 
              cpop=['firename'], saveTo='./', saveName='after_union', saveOnly=True)

In [272]:
hve_onfire_current.drop('geometry', axis=1).to_csv('/Users/jianglongli/Desktop/workbook/data/hve_onfire_current.csv', 
                                           index=False)

### Matching with zipcode & county shapfiel

In [14]:
def download_tiger_shp(url_file, folder_create):
#     folder_create = '%s/zipshpfile' % folder_root
    if not os.path.exists(folder_create):
        os.mkdir(folder_create)
    zipfile = wget.download(url_file, 
                  out=folder_create)
    with zipfile.ZipFile(zfile, "r") as zip_ref: 
        zip_ref.extractall(path=folder_create) 

In [340]:
folder_create = '%s/zipshpfile' % folder_root
if not os.path.exists(folder_create):
    os.mkdir(folder_create)
zipfile = wget.download('ftp://ftp2.census.gov/geo/tiger/TIGER2017/ZCTA5/tl_2017_us_zcta510.zip', 
              out=folder_create)
with zipfile.ZipFile(zfile, "r") as zip_ref: 
    zip_ref.extractall(path=folder_create) 

'/Users/jianglongli/Desktop/workbook/data//tl_2017_us_zcta510.zip'

In [ ]:
folder_create = '%s/cntyshpfile' % folder_root
if not os.path.exists(folder_create):
    os.mkdir(folder_create)
zipfile = wget.download('ftp://ftp2.census.gov/geo/tiger/TIGER2017/COUNTY/tl_2017_us_county.zip', 
              out=folder_create)
with zipfile.ZipFile(cntyfile, "r") as zip_ref: 
    zip_ref.extractall(path=folder_create) 

In [9]:
%%time
gdfzip = readin_shapefile('%s/zipshpfile/tl_2017_us_zcta510.shp' % folder_root)
gdfcnty = readin_shapefile('%s/cntyshpfile/tl_2017_us_county.shp' % folder_root)

CPU times: user 8min 38s, sys: 3.62 s, total: 8min 42s
Wall time: 8min 42s


In [10]:
%%time
result_zip = gpd.sjoin(gdf_union, gdfzip, how='left')

CPU times: user 8.26 s, sys: 59.7 ms, total: 8.32 s
Wall time: 8.35 s


In [11]:
%%time
result_cnty = gpd.sjoin(gdf_union, gdfcnty, how='left')

CPU times: user 1.35 s, sys: 23.9 ms, total: 1.38 s
Wall time: 1.38 s


### Prepare fires map data

In [12]:
# create houses data for mapping
houses = result.drop_duplicates(subset=['zip', 'address'])
houses.loc[:,'color'] = np.where(houses.firename.isnull(), 'grey', 'red')

# create all impacted counties, zips
fire_select = gdf_union
county_select = gdfcnty[gdfcnty.geoid.isin(result_cnty.geoid.unique())]
zip_select = gdfzip[gdfzip.geoid10.isin(result_zip.geoid10.unique())]

# create hve impacted fires data
gdf_hvefire = gdf_union[gdf_union.firename.isin(result.firename.unique())]

# create FEMA counties
fema_counties = ['Butte', 'Lake', 'Mendocino', 'Napa', 'Nevada', 'Sonoma', 'Yuba']
gdf_fema = gdfcnty[gdfcnty.name.isin(fema_counties) & (gdfcnty.statefp == '06')]

# get FEMA turned off zips
zipall = pd.read_excel('/Users/jianglongli/Downloads/Copy of Wildfire_zip_code_1017_2017.xls')
zipall = zipall[zipall.COUNTY_NAME != 'ORANGE']
zipall.loc[:, 'zipcode'] = zipall.zipcode.astype(str)
gdf_femazips = gdfzip[gdfzip.geoid10.isin(zipall.zipcode.unique())]

### Mapping

In [13]:
gdfs = {'all fires': (fire_select, 'firename', '#ff0000') # red 
       ,'all impacted counties': (county_select, 'namelsad', '#f7f327') # yellow
       ,'all impacted zipcode': (zip_select, 'geoid10', '#d1f9f2') # blue
       ,'FEMA counties': (gdf_fema, 'namelsad', '#00ff72')  # purple
       ,'FEMA zips': (gdf_femazips, 'geoid10', '#9a999b')  # grey
       ,'HVE impacted fires': (gdf_hvefire, 'firename', '#ff0000')} # red

map_fires(gdfs, saveName='all_fire_5000', saveOnly=True, saveTo='%s/maps' % folder_root, houses=houses.sample(5000))

# Reference

+ **GeoMAC data, part of USGS, this data provides file polygons**: https://www.geomac.gov/index.shtml 
    - shapefile: https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/
    - data attribute definition: 
        * https://www.nwcg.gov/sites/default/files/stds/WildlandFirePerimeters_definition.pdf
        * https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/historic_fire_data/perimeters_dd83_METADATA.htm
    - curent file perimeter methodology: https://www.geomac.gov/viewer/help/perimeters_active.html    
    - GeoMAC map viewer help and documentation:https://www.geomac.gov/viewer/help/Help.html
    - a 2008 publication about GeoMAC: https://pubs.usgs.gov/ds/612/pdf/ds612.pdf
    - a 2008 GeoMAC user guide: https://webarchive.library.unt.edu/eot2008/20080916004656/http://geomac.gov/pdf/UsersGuide/GeoMAC_UG.pdf


+ **GeoMAC data source from NIFC:** https://www.geomac.gov/about.shtml


+ **NIFC raw data FTP site:** http://ftp.nifc.gov/


+ **USGS**: https://www.usgs.gov/centers/gecsc


+ **Data Basin view of GeoMAC**: https://databasin.org/datasets/6ed18e2a72e74b0d81e14c93d5b46f07


+ **NASA Fire Information for Resource Management System (FIRMS), mostly point data, near real time**: https://earthdata.nasa.gov/earth-observation-data/near-real-time/firms


+ **CA fire org, has google map fire range, but not sure how to get the shapefile**: http://www.calfire.ca.gov/general/firemaps
    - FRAP program from Cal Fire also has fire perimeter data: http://frap.fire.ca.gov/data/frapgisdata-sw-fireperimeters_download
    
    
+ **KML file tutorial**: https://developers.google.com/kml/documentation/kml_tut

# Note

**GeoMAC fire perimeters:**
his layer contains fire perimeters that are submitted to GeoMAC by field offices. The fire perimeters are updated every one or two days, as the data is made available. If we have received no new data, the "expired" layer is not replaced. The layer is replaced as soon as we receive an updated file. Perimeters are usually collected on a daily basis for large fires that are growing. However, there may be gaps in daily coverage.

The GeoMAC team attributes the perimeters using the IRWIN (Integrated Reporting of Wildland-Fire Information) system.

Perimeters are collected in the field by a variety of means, including infrared flights, and by using a GPS unit to map the perimeter. Please NOTE: GeoMAC only displays perimeter data as they are submitted by field offices. Since data are not received for all fires, you may not be able to view perimeters for every fire.

Perimeter data displayed in and delivered by the Geomac application is not the final or official perimeter for any incident and is provided for informational purposes only. The final official perimeter should be obtained from the host unit which can be determined by looking at the Unit Id for any specific fire. The host unit is responsible for producing official and final perimeters for all incidents in their jurisdiction.


**Cal Fire**: 
As part of the California Fire Plan, the Fire and Resource Assessment Program (FRAP) compiles fire perimeters and has established an on-going fire perimeter data capture process in order to update vegetative fuel rank maps. CAL FIRE/FRAP, the USDA Forest Service Region 5 Remote Sensing Lab, the Bureau of Land Management, and the National Park Service jointly develop the comprehensive fire perimeter GIS layer for public and private lands throughout California.

The fire perimeter database represents the most complete digital record of fire perimeters in California. However it is still incomplete in many respects. Fire perimeter database users must exercise caution to avoid inaccurate or erroneous conclusions. For more information on potential errors and their source please review the methodology section of these pages.

# Web Scraping reflection
+ try scrapy (scrapy vs beautifulsoup): https://blog.michaelyin.info/2017/08/10/scrapy-tutorial-1-scrapy-vs-beautiful-soup/

+ scrapy is a framework: https://hexfox.com/p/scrapy-vs-beautifulsoup/